# STEPS TO FOLLOW
### Iterate through each entity (except EPUK and EPL) and for each currency
### 1) Hedge between above and below for each entity != entity_ccy using the entity_ccy as the back-to-back
### 2) Add those amounts of entity_ccy to the corresponding above/below of the entity ccy
### 3) Move the excess of each currency (except entity_ccy) from the entity to EPUK, using the entity_ccy as back-to-back
### 4) Add the amount of these movements of entity_ccy to the entity's exposure in the entity_ccy in the above/below (for each entity and EPUK)
### Now, all the currencies for each entity are hedged, except for the entity_ccy. The entity_ccy exposure isn't moved to EPUK
### 5) Now, for EPUK, hedge between above and below for each entity != entity_ccy using the entity_ccy as the back to back
### 6) Hedge the excess of each currency != entity_ccy in the market with EPUK
### 7) For each entity_ccy in each entity, hedge the excess to the entity_threshold with EPUK (without moving the exposure to EPUK)
### 8) These last hedges need to be moved to EPL so EPUK isn't exposed to that currency, so last step is to do a internal EPUK<->EPL

In [1]:
import pandas as pd
import numpy as np
import queries as qy

In [2]:
import os
from configparser import ConfigParser

def get_config():
    config = ConfigParser()
    config.read(os.path.join("C:/Users/andres.mireles_ebury/Desktop/Projects/FX Exposure/fx_exposure/config.ini"))

    return config

from bq_link import get_bq_link
bq_client, _ = get_bq_link(get_config())

Load all the exposures

In [3]:
balance_date = "2024-04-30"
exposures_entities = bq_client.query(qy.net_exposure.format(date=balance_date)).drop_duplicates()

Set the group threshold and the individual entity threshold (in the future, these can be weighted)

The total exposures by currency

In [4]:
total_exposures = exposures_entities.groupby("currency")[["above_exposure_gbp","below_exposure_gbp","net_exposure_gbp"]].sum()

Use only the entitiies with high exposures

In [5]:
group_threshold = 5e6

In [6]:
large_exposure_currencies = total_exposures[
    (np.abs(total_exposures.above_exposure_gbp)>=group_threshold)|
    (np.abs(total_exposures.below_exposure_gbp)>=group_threshold)|
    (np.abs(total_exposures.net_exposure_gbp)>=group_threshold)
].index

In [7]:
large_exposure_currencies

Index(['AED', 'AUD', 'CAD', 'CHF', 'EUR', 'GBP', 'HKD', 'PLN', 'USD'], dtype='object', name='currency')

In [8]:
exposures_entities = exposures_entities[exposures_entities.currency.isin(large_exposure_currencies)].reset_index(drop=True)

In [9]:
exposures_entities = exposures_entities.set_index(["currency","entity","entity_ccy"])

In [10]:
exposures_entities = exposures_entities[[c for c in exposures_entities if "local" not in c]]

# THIS PART IS NOT CORRECT (IT DOESN'T ALLOW US TO SEE THE FULL EXPOSURE) AND NOT USEFUL

Only the entities with large exposures to each currency

In [11]:
entity_threshold = group_threshold / (2 * exposures_entities.index.get_level_values(1).unique().shape[0])

In [12]:
# large_exposure_entities_currencies = exposures_entities[
#     (np.abs(exposures_entities.above_exposure_gbp)>=entity_threshold)|
#     (np.abs(exposures_entities.below_exposure_gbp)>=entity_threshold)|
#     (np.abs(exposures_entities.net_exposure_gbp)>=entity_threshold)
# ].index

In [13]:
# exposures_entities = exposures_entities.loc[large_exposure_entities_currencies]

In [14]:
def net_exposure(currency,entity,entity_ccy, above,above_entity_ccy,below,below_entity_ccy,threshold,exclude_epuk=True,exclude_epl=True):

    if above == below == 0 or (exclude_epl and entity == "EPL") or (exclude_epuk and entity == "EPUK") or currency == entity_ccy:
        return above, above_entity_ccy, below, below_entity_ccy, above+below, above_entity_ccy+below_entity_ccy, 0, 0, np.nan, np.nan, 0, 0, np.nan

    # Rate to convert gbp to local ccy
    if below != 0:
        rate_to_entity_ccy = below_entity_ccy / below
    else:
        rate_to_entity_ccy = above_entity_ccy / above


    # Signs of the below to move the exposure (if below is 0, then just the opposite of the sign of the above)
    if below != 0:
        sign_below = below / abs(below)
    else:
        sign_below = -1 * above / abs(above)

    # The internal change that we need
    internal_change = np.maximum(abs(below) - threshold,0)

    internal_change_entity_ccy = internal_change * rate_to_entity_ccy

    # The resulting exposures
    new_above = above + sign_below*internal_change
    new_above_entity_ccy = above_entity_ccy + sign_below*internal_change_entity_ccy
    new_below = below - sign_below*internal_change
    new_below_entity_ccy = below_entity_ccy - sign_below*internal_change_entity_ccy

    # The direction of the internal entity hedges
    below_lhs_rhs = np.where(
        internal_change != 0,
        np.where(sign_below>0,"LHS","RHS"),
        np.nan
    )
    above_lhs_rhs = np.where(
        internal_change != 0,
        np.where(sign_below>0,"RHS","LHS"),
        np.nan
    )

    # What we are going to move from the above of the entity to the above in epuk (the threhold here is 2 times the threshold for covering above/below)
    above_to_epuk = np.maximum(abs(new_above) - threshold * 2,0)

    above_to_epuk_entity_ccy = above_to_epuk * rate_to_entity_ccy
    
    # The direction of the above to epuk hedges (if above = 0, we don't move anything)
    above_to_epuk_lhs_rhs = np.where(
        above_to_epuk != 0,
        np.where(new_above/abs(new_above)>0,"LHS","RHS"),
        np.nan
    )

    # Update the new above of that entity reducing what we move to epuk
    new_above = new_above - np.where(
        new_above!=0,
        new_above/abs(new_above)*above_to_epuk,
        0
    )
    new_above_entity_ccy = new_above_entity_ccy - np.where(
        new_above!=0,
        new_above_entity_ccy/abs(new_above_entity_ccy)*above_to_epuk_entity_ccy,
        0
    )

    # New net
    new_net = new_below + new_above
    new_net_entity_ccy = new_below_entity_ccy + new_above_entity_ccy
    


    return new_above, new_above_entity_ccy, new_below, new_below_entity_ccy, new_net, new_net_entity_ccy, internal_change, internal_change_entity_ccy, above_lhs_rhs, below_lhs_rhs, above_to_epuk, above_to_epuk_entity_ccy, above_to_epuk_lhs_rhs


In [15]:
exposures_entities.head()

,,,above_exposure_gbp,above_exposure_entity_ccy,below_exposure_gbp,below_exposure_entity_ccy,net_exposure_gbp,net_exposure_entity_ccy
currency,entity,entity_ccy,,,,,,
CHF,EPUK,GBP,-4355491.77,-4.355492e+06,3.180092e+06,3.180092e+06,-1.175400e+06,-1.175400e+06
USD,EPCHL,CLP,77.66,9.284083e+04,1.435878e+03,1.716528e+06,1.513538e+03,1.809369e+06
AUD,EPHK,HKD,543.06,5.314251e+03,2.760516e+04,2.701392e+05,2.814822e+04,2.754535e+05
AED,EPCA,CAD,0.00,0.000000e+00,7.616089e+04,1.310129e+05,7.616089e+04,1.310129e+05
GBP,EPL,GBP,0.00,0.000000e+00,6.285352e+07,6.285352e+07,6.285352e+07,6.285352e+07


## Step 1)

### For each entity except EPUK & EPL, we hedge above/below and move out of each ccy except entity_ccy the remaining exposure

In [16]:
import warnings

warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

exposures_entities_net = pd.DataFrame()

for entity in exposures_entities.index.get_level_values(1).unique():
    
    exposure_ent = exposures_entities.xs(entity, level=1)

    exposure_ent[
        [
            'new_above_exposure_gbp', 
            'new_above_exposure_entity_ccy', 
            'new_below_exposure_gbp', 
            'new_below_exposure_entity_ccy', 
            'new_net_exposure_gbp', 
            'new_net_exposure_entity_ccy', 
            'internal_change_gbp', 
            'internal_change_entity_ccy', 
            'above_lhs_rhs', 
            'below_lhs_rhs', 
            'above_to_epuk_gbp', 
            'above_to_epuk_entity_ccy', 
            'above_to_epuk_lhs_rhs'
        ]
    ] = exposure_ent.apply(
        lambda row: pd.Series(
            net_exposure(
                row.name[0],
                entity, 
                row.name[1], 
                row['above_exposure_gbp'], 
                row['above_exposure_entity_ccy'], 
                row['below_exposure_gbp'], 
                row['below_exposure_entity_ccy'], 
                entity_threshold,
                exclude_epuk=True,
                exclude_epl=True
            )
        ),
        axis=1
    )

    exposure_ent["entity"] = entity
    exposure_ent = exposure_ent.reset_index().set_index(["currency","entity","entity_ccy"])
    
    exposures_entities_net = pd.concat([exposures_entities_net,exposure_ent])
    

C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in double_scalars


In [17]:
exposures_entities_net.xs("EPBE",level=1)[[c for c in exposure_ent.columns if "ccy" not in c]]

,,above_exposure_gbp,below_exposure_gbp,net_exposure_gbp,new_above_exposure_gbp,new_below_exposure_gbp,new_net_exposure_gbp,internal_change_gbp,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_lhs_rhs
currency,entity_ccy,,,,,,,,,,,
GBP,EUR,8.275989e+05,7.409683e+07,7.492443e+07,2.941176e+05,1.470588e+05,441176.470588,7.394978e+07,RHS,LHS,7.448326e+07,LHS
HKD,EUR,1.164153e-10,-2.467729e+03,-2.467729e+03,1.164153e-10,-2.467729e+03,-2467.729208,0.000000e+00,nan,nan,0.000000e+00,nan
CAD,EUR,7.970070e+03,1.165937e+05,1.245638e+05,7.970070e+03,1.165937e+05,124563.787083,0.000000e+00,nan,nan,0.000000e+00,nan
EUR,EUR,6.923909e+07,-6.897756e+07,2.615338e+05,6.923909e+07,-6.897756e+07,261533.793454,0.000000e+00,NaN,NaN,0.000000e+00,NaN
USD,EUR,5.756789e+06,8.981906e+07,9.557585e+07,2.941176e+05,1.470588e+05,441176.470588,8.967200e+07,RHS,LHS,9.513467e+07,LHS
CHF,EUR,3.507185e+05,-4.712774e+06,-4.362055e+06,-2.941176e+05,-1.470588e+05,-441176.470588,4.565715e+06,LHS,RHS,3.920879e+06,RHS
AED,EUR,1.806255e+04,-1.438481e+05,-1.257856e+05,1.806255e+04,-1.438481e+05,-125785.575036,0.000000e+00,nan,nan,0.000000e+00,nan
AUD,EUR,1.574927e+04,-4.826752e+05,-4.669259e+05,-2.941176e+05,-1.470588e+05,-441176.470588,3.356163e+05,LHS,RHS,2.574942e+04,RHS
PLN,EUR,2.871657e+06,-1.339182e+07,-1.052016e+07,-2.941176e+05,-1.470588e+05,-441176.470588,1.324476e+07,LHS,RHS,1.007898e+07,RHS


## Steps 2) & 4) (except EPUK)

In [18]:
exposures_entities_net_2 = pd.DataFrame()

for entity in exposures_entities_net.index.get_level_values(1).unique():

    exposure_ent = exposures_entities_net.xs(entity, level=1)

    if entity not in ("EPUK","EPL"):
        
        # If we don't have already exposure for the entity_ccy, we create it full of 0
        if exposure_ent.index.get_level_values(1).unique().item() not in exposure_ent.index.get_level_values(0):

            exposure_ent = pd.concat(
                [
                    exposure_ent,
                    pd.DataFrame(
                        np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.nan,np.nan,0,0,np.nan]).reshape(-1,1).T, 
                        index=pd.MultiIndex.from_tuples(
                            [(exposure_ent.index.get_level_values(1).unique().item(),exposure_ent.index.get_level_values(1).unique().item())], 
                            names=exposure_ent.index.names
                            ),
                        columns=exposure_ent.columns
                    )
                ]
            )
            
        # Locate the entity_ccy exposure to add the exposure from the internal changes (to the above and below) and from the hedges to EPUK to the above
        exposure_ent.loc[
            exposure_ent.index.get_level_values(0)==exposure_ent.index.get_level_values(1).unique().item(),
        [
            "new_above_exposure_gbp",
            "new_above_exposure_entity_ccy",
        ]] += (np.where(
                exposure_ent[["above_lhs_rhs"]]=="LHS",
                exposure_ent[["internal_change_gbp","internal_change_entity_ccy"]],
                -1 * exposure_ent[["internal_change_gbp","internal_change_entity_ccy"]]
        ).sum(axis=0) + np.where(
                exposure_ent[["above_to_epuk_lhs_rhs"]]=="LHS",
                exposure_ent[["above_to_epuk_gbp","above_to_epuk_entity_ccy"]],
                -1 * exposure_ent[["above_to_epuk_gbp","above_to_epuk_entity_ccy"]]
        ).sum(axis=0))

        exposure_ent.loc[
            exposure_ent.index.get_level_values(0)==exposure_ent.index.get_level_values(1).unique().item(),
        [
            "new_below_exposure_gbp",
            "new_below_exposure_entity_ccy",
        ]] += np.where(
                exposure_ent[["below_lhs_rhs"]]=="LHS",
                exposure_ent[["internal_change_gbp","internal_change_entity_ccy"]],
                -1 * exposure_ent[["internal_change_gbp","internal_change_entity_ccy"]]
        ).sum(axis=0)

        # The net exposure of the entity_ccy only changes due to the movement of exposure to EPUK
        exposure_ent.loc[
            exposure_ent.index.get_level_values(0)==exposure_ent.index.get_level_values(1).unique().item(),
        [
            "new_net_exposure_gbp",
            "new_net_exposure_entity_ccy",
        ]] += np.where(
                exposure_ent[["above_to_epuk_lhs_rhs"]]=="LHS",
                exposure_ent[["above_to_epuk_gbp","above_to_epuk_entity_ccy"]],
                -1 * exposure_ent[["above_to_epuk_gbp","above_to_epuk_entity_ccy"]]
        ).sum(axis=0)



    exposure_ent["entity"] = entity
    exposure_ent = exposure_ent.reset_index().set_index(["currency","entity","entity_ccy"])

    exposures_entities_net_2 = pd.concat([exposures_entities_net_2,exposure_ent])



## Step 3)

In [19]:
exposures_entities_not_entity_ccy = exposures_entities_net_2.copy()
exposures_entities_not_entity_ccy = exposures_entities_not_entity_ccy[exposures_entities_not_entity_ccy.index.get_level_values(0) != exposures_entities_not_entity_ccy.index.get_level_values(2)]

In [20]:
exposures_to_epuk_entities_not_entity_ccy = exposures_entities_not_entity_ccy.reset_index()
exposures_to_epuk_entities_not_entity_ccy[
    [
        "above_to_epuk_gbp",
        "above_to_epuk_entity_ccy"
    ]
] = np.where(
    exposures_to_epuk_entities_not_entity_ccy[["above_to_epuk_lhs_rhs"]] == "LHS",
    exposures_to_epuk_entities_not_entity_ccy[
        [
            "above_to_epuk_gbp",
            "above_to_epuk_entity_ccy"
        ]
    ],
    -1 * exposures_to_epuk_entities_not_entity_ccy[
        [
            "above_to_epuk_gbp",
            "above_to_epuk_entity_ccy"
        ]
    ]
)

# The change that we need to add by currency
exposures_to_epuk = exposures_to_epuk_entities_not_entity_ccy.groupby("currency")[[
            "above_to_epuk_gbp",
            "above_to_epuk_entity_ccy"
]].sum().reset_index()

# Add these required fields
exposures_to_epuk[["entity","entity_ccy"]] = ["EPUK","GBP"]

exposures_to_epuk = exposures_to_epuk.set_index(["currency","entity","entity_ccy"])

# Add the exposures to EPUK (to the above and to the net)
exposures_entities_net_3 = exposures_entities_net_2.copy()
exposures_entities_net_3.loc[
    ((exposures_entities_net_2.index.get_level_values(1)=="EPUK")&(exposures_entities_net_2.index.get_level_values(2)=="GBP")),
    ["new_above_exposure_gbp","new_above_exposure_entity_ccy"]] += exposures_to_epuk.values
exposures_entities_net_3.loc[
    ((exposures_entities_net_2.index.get_level_values(1)=="EPUK")&(exposures_entities_net_2.index.get_level_values(2)=="GBP")),
    ["new_net_exposure_gbp","new_net_exposure_entity_ccy"]] += exposures_to_epuk.values

In [21]:
exposures_entities_net_3[exposures_entities_net_3.index.get_level_values(1)=="EPUK"][[c for c in exposure_ent.columns if "ccy" not in c]]

,,,above_exposure_gbp,below_exposure_gbp,net_exposure_gbp,new_above_exposure_gbp,new_below_exposure_gbp,new_net_exposure_gbp,internal_change_gbp,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_lhs_rhs
currency,entity,entity_ccy,,,,,,,,,,,
CHF,EPUK,GBP,-4.355492e+06,3.180092e+06,-1.175400e+06,-6.155334e+06,3.180092e+06,-2.975242e+06,0.0,NaN,NaN,0.0,NaN
EUR,EPUK,GBP,-8.015238e+07,1.409674e+08,6.081498e+07,-7.977320e+07,1.409674e+08,6.119415e+07,0.0,NaN,NaN,0.0,NaN
AED,EPUK,GBP,-2.200000e-01,-9.873862e+06,-9.873862e+06,-2.200000e-01,-9.873862e+06,-9.873862e+06,0.0,NaN,NaN,0.0,NaN
AUD,EPUK,GBP,-2.590651e+07,1.365527e+07,-1.225125e+07,-2.982739e+07,1.365527e+07,-1.617213e+07,0.0,NaN,NaN,0.0,NaN
HKD,EPUK,GBP,3.183231e-12,-2.626989e+06,-2.626989e+06,-1.826849e+07,-2.626989e+06,-2.089548e+07,0.0,NaN,NaN,0.0,NaN
PLN,EPUK,GBP,-9.311577e+05,5.686571e+05,-3.625006e+05,7.606020e+07,5.686571e+05,7.662886e+07,0.0,NaN,NaN,0.0,NaN
USD,EPUK,GBP,-2.529958e+06,1.179007e+07,9.260114e+06,-2.529958e+06,1.179007e+07,9.260114e+06,0.0,NaN,NaN,0.0,NaN
CAD,EPUK,GBP,5.865700e+02,-3.635368e+06,-3.634782e+06,-1.007840e+07,-3.635368e+06,-1.371376e+07,0.0,NaN,NaN,0.0,NaN
GBP,EPUK,GBP,1.407771e+08,-2.020161e+08,-6.123892e+07,2.247385e+08,-2.020161e+08,2.272242e+07,0.0,NaN,NaN,0.0,NaN


## Step 4) (for EPUK)

In [22]:
# Here, we susbtract the amount instead of adding it
exposures_to_epuk_entity_ccy = (-1 * exposures_to_epuk_entities_not_entity_ccy.groupby(by=["entity_ccy"])[[
            "above_to_epuk_gbp",
            "above_to_epuk_entity_ccy"
]].sum()).reset_index().rename(columns={"entity_ccy":"currency"})

exposures_to_epuk_entity_ccy[["entity","entity_ccy"]] = ["EPUK","GBP"]

exposures_to_epuk_entity_ccy = exposures_to_epuk_entity_ccy.set_index(["currency","entity","entity_ccy"])

# First we fill the gaps in currencies in EPUK
exposures_entities_net_4 = exposures_entities_net_3.copy()

for currency in exposures_to_epuk_entity_ccy.index.get_level_values(0):
    if currency not in exposures_entities_net_4.xs("EPUK",level=1).index.get_level_values(0):
        exposures_entities_net_4 = pd.concat(
                [
                    exposures_entities_net_4,
                    pd.DataFrame(
                        np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.nan,np.nan,0,0,np.nan]).reshape(-1,1).T, 
                        index=pd.MultiIndex.from_tuples(
                            [(currency,"EPUK","GBP")], 
                            names=exposures_entities_net_4.index.names
                            ),
                        columns=exposures_entities_net_4.columns
                    )
                ]
            )
        
# We need to fill also the the gaps for the exposures in the currencies that we are going to add
for currency in exposures_entities_net_4.xs("EPUK",level=1).index.get_level_values(0):
    if currency not in exposures_to_epuk_entity_ccy.index.get_level_values(0):
        exposures_to_epuk_entity_ccy = pd.concat(
                [
                    exposures_to_epuk_entity_ccy,
                    pd.DataFrame(
                        np.array([0,0]).reshape(-1,1).T, 
                        index=pd.MultiIndex.from_tuples(
                            [(currency,"EPUK","GBP")], 
                            names=exposures_to_epuk_entity_ccy.index.names
                            ),
                        columns=exposures_to_epuk_entity_ccy.columns
                    )
                ]
            )

# Add the exposures to EPUK (to the above and to the net)
exposures_to_epuk_entity_ccy.columns = ["new_above_exposure_gbp","new_above_exposure_entity_ccy"]
exposures_entities_net_4.loc[
    ((exposures_entities_net_4.index.get_level_values(1)=="EPUK")&(exposures_entities_net_4.index.get_level_values(2)=="GBP")),
    ["new_above_exposure_gbp","new_above_exposure_entity_ccy"]] += exposures_to_epuk_entity_ccy

exposures_to_epuk_entity_ccy.columns = ["new_net_exposure_gbp","new_net_exposure_entity_ccy"]
exposures_entities_net_4.loc[
    ((exposures_entities_net_4.index.get_level_values(1)=="EPUK")&(exposures_entities_net_4.index.get_level_values(2)=="GBP")),
    ["new_net_exposure_gbp","new_net_exposure_entity_ccy"]] += exposures_to_epuk_entity_ccy

In [23]:
exposures_entities_net_4.xs("EPUK",level=1)[[c for c in exposure_ent.columns if "ccy" not in c]]

,,above_exposure_gbp,below_exposure_gbp,net_exposure_gbp,new_above_exposure_gbp,new_below_exposure_gbp,new_net_exposure_gbp,internal_change_gbp,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_lhs_rhs
currency,entity_ccy,,,,,,,,,,,
CHF,GBP,-4.355492e+06,3.180092e+06,-1.175400e+06,-1.282222e+07,3.180092e+06,-9.642131e+06,0.0,NaN,NaN,0.0,NaN
EUR,GBP,-8.015238e+07,1.409674e+08,6.081498e+07,-2.353655e+08,1.409674e+08,-9.439817e+07,0.0,NaN,NaN,0.0,NaN
AED,GBP,-2.200000e-01,-9.873862e+06,-9.873862e+06,-2.200000e-01,-9.873862e+06,-9.873862e+06,0.0,NaN,NaN,0.0,NaN
AUD,GBP,-2.590651e+07,1.365527e+07,-1.225125e+07,-3.004505e+07,1.365527e+07,-1.638978e+07,0.0,NaN,NaN,0.0,NaN
HKD,GBP,3.183231e-12,-2.626989e+06,-2.626989e+06,-2.196647e+07,-2.626989e+06,-2.459346e+07,0.0,NaN,NaN,0.0,NaN
PLN,GBP,-9.311577e+05,5.686571e+05,-3.625006e+05,7.606020e+07,5.686571e+05,7.662886e+07,0.0,NaN,NaN,0.0,NaN
USD,GBP,-2.529958e+06,1.179007e+07,9.260114e+06,-1.799984e+06,1.179007e+07,9.990088e+06,0.0,NaN,NaN,0.0,NaN
CAD,GBP,5.865700e+02,-3.635368e+06,-3.634782e+06,-1.315992e+07,-3.635368e+06,-1.679529e+07,0.0,NaN,NaN,0.0,NaN
GBP,GBP,1.407771e+08,-2.020161e+08,-6.123892e+07,2.656287e+08,-2.020161e+08,6.361266e+07,0.0,NaN,NaN,0.0,NaN


In [24]:
exposures_entities.net_exposure_gbp.sum()

10725717.776619824

In [25]:
exposures_entities_net_4.net_exposure_gbp.sum()

10725717.776619818

In [26]:
exposures_entities_net_4.new_net_exposure_gbp.sum()

10725717.776619798

## Step 5) & 6)

In [27]:
exposures_entities_net_5 = exposures_entities_net_4.copy().xs("EPUK",level=1)

In [28]:
import warnings

warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

entity = "EPUK"

exposures_entities_net_5[
    [
        'new_above_exposure_gbp', 
        'new_above_exposure_entity_ccy', 
        'new_below_exposure_gbp', 
        'new_below_exposure_entity_ccy', 
        'new_net_exposure_gbp', 
        'new_net_exposure_entity_ccy', 
        'internal_change_gbp', 
        'internal_change_entity_ccy', 
        'above_lhs_rhs', 
        'below_lhs_rhs', 
        'above_to_epuk_gbp', 
        'above_to_epuk_entity_ccy', 
        'above_to_epuk_lhs_rhs'
    ]
] = exposures_entities_net_5.apply(
    lambda row: pd.Series(
        net_exposure(
            row.name[0],
            entity, 
            row.name[1], 
            row['new_above_exposure_gbp'],
            row['new_above_exposure_entity_ccy'], 
            row['new_below_exposure_gbp'],
            row['new_below_exposure_entity_ccy'], 
            entity_threshold,
            exclude_epuk=False,
            exclude_epl=True
        )
    ),
    axis=1
)


In [29]:
exposures_entities_net_5

,,above_exposure_gbp,above_exposure_entity_ccy,below_exposure_gbp,below_exposure_entity_ccy,net_exposure_gbp,net_exposure_entity_ccy,new_above_exposure_gbp,new_above_exposure_entity_ccy,new_below_exposure_gbp,new_below_exposure_entity_ccy,new_net_exposure_gbp,new_net_exposure_entity_ccy,internal_change_gbp,internal_change_entity_ccy,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_entity_ccy,above_to_epuk_lhs_rhs
currency,entity_ccy,,,,,,,,,,,,,,,,,,,
CHF,GBP,-4.355492e+06,-4.355492e+06,3.180092e+06,3.180092e+06,-1.175400e+06,-1.175400e+06,-2.941176e+05,-1.732778e+06,1.470588e+05,1.470588e+05,-1.470588e+05,-1.585719e+06,3.033033e+06,3.033033e+06,RHS,LHS,9.495072e+06,9.495072e+06,RHS
EUR,GBP,-8.015238e+07,-8.015238e+07,1.409674e+08,1.409674e+08,6.081498e+07,6.081498e+07,-2.941176e+05,-2.691990e+07,1.470588e+05,1.470588e+05,-1.470588e+05,-2.677284e+07,1.408203e+08,1.408203e+08,RHS,LHS,9.425111e+07,9.425111e+07,RHS
AED,GBP,-2.200000e-01,-2.176026e-01,-9.873862e+06,-9.873862e+06,-9.873862e+06,-9.873862e+06,-2.941176e+05,-2.941176e+05,-1.470588e+05,-1.470588e+05,-4.411765e+05,-4.411765e+05,9.726803e+06,9.726803e+06,LHS,RHS,9.432686e+06,9.432686e+06,RHS
AUD,GBP,-2.590651e+07,-2.590651e+07,1.365527e+07,1.365527e+07,-1.225125e+07,-1.225125e+07,-2.941176e+05,-1.166987e+06,1.470588e+05,1.470588e+05,-1.470588e+05,-1.019928e+06,1.350821e+07,1.350821e+07,RHS,LHS,1.624272e+07,1.624272e+07,RHS
HKD,GBP,3.183231e-12,1.364242e-12,-2.626989e+06,-2.626989e+06,-2.626989e+06,-2.626989e+06,-2.941176e+05,-3.151174e+07,-1.470588e+05,-1.470588e+05,-4.411765e+05,-3.165880e+07,2.479930e+06,2.479930e+06,LHS,RHS,2.415228e+07,2.415228e+07,RHS
PLN,GBP,-9.311577e+05,-9.311577e+05,5.686571e+05,5.686570e+05,-3.625006e+05,-3.625007e+05,2.941176e+05,7.094342e+07,1.470588e+05,1.470588e+05,4.411765e+05,7.109048e+07,4.215982e+05,4.215982e+05,RHS,LHS,7.618768e+07,7.618768e+07,LHS
USD,GBP,-2.529958e+06,-2.529958e+06,1.179007e+07,1.179007e+07,9.260114e+06,9.260114e+06,2.941176e+05,4.774868e+05,1.470588e+05,1.470588e+05,4.411765e+05,6.245456e+05,1.164301e+07,1.164301e+07,RHS,LHS,9.548912e+06,9.548912e+06,LHS
CAD,GBP,5.865700e+02,5.865729e+02,-3.635368e+06,-3.635368e+06,-3.634782e+06,-3.634782e+06,-2.941176e+05,-4.237953e+06,-1.470588e+05,-1.470588e+05,-4.411765e+05,-4.385012e+06,3.488310e+06,3.488310e+06,LHS,RHS,1.635411e+07,1.635411e+07,RHS
GBP,GBP,1.407771e+08,1.407771e+08,-2.020161e+08,-2.020161e+08,-6.123892e+07,-6.123892e+07,2.656287e+08,2.586317e+08,-2.020161e+08,-2.020161e+08,6.361266e+07,5.661562e+07,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,NaN


### Step 5.1) Add the entity_ccy amount of the back-to-back from the internal and also the ones from the external

In [30]:
# Above
exposures_entities_net_5.loc[
    exposures_entities_net_5.index.get_level_values(0)=="GBP",
    ["new_above_exposure_gbp","new_above_exposure_entity_ccy"]
] += (np.where(
    exposures_entities_net_5[["above_lhs_rhs"]]=="LHS",
    exposures_entities_net_5[["internal_change_gbp","internal_change_entity_ccy"]],
    -1 * exposures_entities_net_5[["internal_change_gbp","internal_change_entity_ccy"]]
).sum(axis=0) + np.where(
    exposures_entities_net_5[["above_to_epuk_lhs_rhs"]]=="LHS",
    exposures_entities_net_5[["above_to_epuk_gbp","above_to_epuk_entity_ccy"]],
    -1 * exposures_entities_net_5[["above_to_epuk_gbp","above_to_epuk_entity_ccy"]]
).sum(axis=0))

# Below
exposures_entities_net_5.loc[
    exposures_entities_net_5.index.get_level_values(0)=="GBP",
    ["new_below_exposure_gbp","new_below_exposure_entity_ccy"]
] += (np.where(
    exposures_entities_net_5[["below_lhs_rhs"]]=="LHS",
    exposures_entities_net_5[["internal_change_gbp","internal_change_entity_ccy"]],
    -1 * exposures_entities_net_5[["internal_change_gbp","internal_change_entity_ccy"]]
).sum(axis=0))

# Net (only changes due to the external movement of exposure)
exposures_entities_net_5.loc[
    exposures_entities_net_5.index.get_level_values(0)=="GBP",
    ["new_net_exposure_gbp","new_net_exposure_entity_ccy"]
] += np.where(
    exposures_entities_net_5[["above_to_epuk_lhs_rhs"]]=="LHS",
    exposures_entities_net_5[["above_to_epuk_gbp","above_to_epuk_entity_ccy"]],
    -1 * exposures_entities_net_5[["above_to_epuk_gbp","above_to_epuk_entity_ccy"]]
).sum(axis=0)


# Reshape it to the standard shape
exposures_entities_net_5 = exposures_entities_net_5.reset_index()
exposures_entities_net_5["entity"] = "EPUK"

exposures_entities_net_5 = exposures_entities_net_5.set_index(["currency","entity","entity_ccy"]) 

Put all together

In [31]:
exposures_entities_net_6 = pd.concat([exposures_entities_net_4[exposures_entities_net_4.index.get_level_values(1)!="EPUK"],exposures_entities_net_5])

## Step 7

#### In this step, note that we can't add the exposure in entity_ccy since we are going to mix different currencies (since each entity has its own entity ccy)

In [32]:
exposures_epl = exposures_entities_net_6[exposures_entities_net_6.index.get_level_values(1)=="EPL"]

In [33]:
exposures_entities_currency_to_epl = exposures_entities_net_6[exposures_entities_net_6.index.get_level_values(0)==exposures_entities_net_6.index.get_level_values(2)].reset_index()
exposures_entities_currency_to_epl["entity"] = "EPL"
exposures_entities_currency_to_epl["entity_ccy"] = "GBP"
exposures_entities_currency_to_epl = exposures_entities_currency_to_epl.set_index(["currency","entity","entity_ccy"]).groupby(["currency","entity","entity_ccy"]).sum()
exposures_entities_currency_to_epl[["above_lhs_rhs","below_lhs_rhs","above_to_epuk_lhs_rhs"]] = np.nan

exposures_entities_currency_to_epl = exposures_entities_currency_to_epl[exposures_entities_net_6.columns]

First, we fill the currencies that we currently don't have in EPL but that we are going to move there

In [34]:
for currency in exposures_entities_currency_to_epl.index.get_level_values(0).unique():
    if currency not in exposures_epl.index.get_level_values(0).unique():

        exposures_epl = pd.concat(
            [
                exposures_epl,
                pd.DataFrame(
                    np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.nan,np.nan,0,0,np.nan]).reshape(-1,1).T, 
                    index=pd.MultiIndex.from_tuples(
                        [(currency,"EPL","GBP")], 
                        names=exposures_epl.index.names
                        ),
                    columns=exposures_epl.columns
                )
            ]
        )

for currency in exposures_epl.index.get_level_values(0).unique():
    if currency not in exposures_entities_currency_to_epl.index.get_level_values(0).unique():

        exposures_entities_currency_to_epl = pd.concat(
            [
                exposures_entities_currency_to_epl,
                pd.DataFrame(
                    np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.nan,np.nan,0,0,np.nan]).reshape(-1,1).T, 
                    index=pd.MultiIndex.from_tuples(
                        [(currency,"EPL","GBP")], 
                        names=exposures_entities_currency_to_epl.index.names
                        ),
                    columns=exposures_entities_currency_to_epl.columns
                )
            ]
        )

Add (fictitiously) the exposures of each entity_ccy to EPL

In [35]:
exposures_epl[exposures_epl.columns[9:]][[c for c in exposures_epl[exposures_epl.columns[9:]].columns if "entity_ccy" not in c]] = \
    exposures_epl[exposures_epl.columns[9:]][[c for c in exposures_epl[exposures_epl.columns[9:]].columns if "entity_ccy" not in c]] + \
    exposures_entities_currency_to_epl[exposures_entities_currency_to_epl.columns[9:]][[c for c in exposures_entities_currency_to_epl[exposures_entities_currency_to_epl.columns[9:]].columns if "entity_ccy" not in c]]

In [36]:
exposures_epl.new_net_exposure_gbp.sum()

-90783264.64572895

In [37]:
exposures_epl

,,,above_exposure_gbp,above_exposure_entity_ccy,below_exposure_gbp,below_exposure_entity_ccy,net_exposure_gbp,net_exposure_entity_ccy,new_above_exposure_gbp,new_above_exposure_entity_ccy,new_below_exposure_gbp,new_below_exposure_entity_ccy,new_net_exposure_gbp,new_net_exposure_entity_ccy,internal_change_gbp,internal_change_entity_ccy,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_entity_ccy,above_to_epuk_lhs_rhs
currency,entity,entity_ccy,,,,,,,,,,,,,,,,,,,
GBP,EPL,GBP,0.0,0.0,6.285352e+07,6.285352e+07,6.285352e+07,6.285352e+07,0.0,0.0,6.285352e+07,6.285352e+07,6.285352e+07,6.285352e+07,0.0,0.0,NaN,NaN,0.0,0.0,NaN
HKD,EPL,GBP,0.0,0.0,-3.064122e+06,-3.064122e+06,-3.064122e+06,-3.064122e+06,0.0,0.0,-3.064122e+06,-3.064122e+06,-3.064122e+06,-3.064122e+06,0.0,0.0,NaN,NaN,0.0,0.0,NaN
EUR,EPL,GBP,0.0,0.0,-4.787437e+07,-4.787437e+07,-4.787437e+07,-4.787437e+07,0.0,0.0,-4.787437e+07,-4.787437e+07,-4.787437e+07,-4.787437e+07,0.0,0.0,NaN,NaN,0.0,0.0,NaN
CAD,EPL,GBP,0.0,0.0,-5.817170e+06,-5.817170e+06,-5.817170e+06,-5.817170e+06,0.0,0.0,-5.817170e+06,-5.817170e+06,-5.817170e+06,-5.817170e+06,0.0,0.0,NaN,NaN,0.0,0.0,NaN
USD,EPL,GBP,0.0,0.0,-9.020751e+07,-9.020751e+07,-9.020751e+07,-9.020751e+07,0.0,0.0,-9.020751e+07,-9.020751e+07,-9.020751e+07,-9.020751e+07,0.0,0.0,NaN,NaN,0.0,0.0,NaN
PLN,EPL,GBP,0.0,0.0,-1.606141e+03,-1.606141e+03,-1.606141e+03,-1.606141e+03,0.0,0.0,-1.606141e+03,-1.606141e+03,-1.606141e+03,-1.606141e+03,0.0,0.0,NaN,NaN,0.0,0.0,NaN
CHF,EPL,GBP,0.0,0.0,-4.440634e+06,-4.440634e+06,-4.440634e+06,-4.440634e+06,0.0,0.0,-4.440634e+06,-4.440634e+06,-4.440634e+06,-4.440634e+06,0.0,0.0,NaN,NaN,0.0,0.0,NaN
AED,EPL,GBP,0.0,0.0,8.250687e+02,8.250687e+02,8.250687e+02,8.250687e+02,0.0,0.0,8.250687e+02,8.250687e+02,8.250687e+02,8.250687e+02,0.0,0.0,NaN,NaN,0.0,0.0,NaN
AUD,EPL,GBP,0.0,0.0,-2.232191e+06,-2.232191e+06,-2.232191e+06,-2.232191e+06,0.0,0.0,-2.232191e+06,-2.232191e+06,-2.232191e+06,-2.232191e+06,0.0,0.0,NaN,NaN,0.0,0.0,NaN


Net between above and below in EPL

In [38]:
import warnings

warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

entity = "EPL"

exposures_epl[
    [
        'new_above_exposure_gbp',
        'new_above_exposure_entity_ccy', 
        'new_below_exposure_gbp',
        'new_below_exposure_entity_ccy', 
        'new_net_exposure_gbp',
        'new_net_exposure_entity_ccy', 
        'internal_change_gbp',
        'internal_change_entity_ccy', 
        'above_lhs_rhs', 
        'below_lhs_rhs', 
        'above_to_epuk_gbp',
        'above_to_epuk_entity_ccy', 
        'above_to_epuk_lhs_rhs'
    ]
] = exposures_epl.apply(
    lambda row: pd.Series(
        net_exposure(
            row.name[0],
            entity, 
            row.name[1], 
            row['new_above_exposure_gbp'],
            row['new_above_exposure_entity_ccy'], 
            row['new_below_exposure_gbp'],
            row['new_below_exposure_entity_ccy'], 
            entity_threshold,
            exclude_epuk=True,
            exclude_epl=False
        )
    ),
    axis=1
)


C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in double_scalars


In [39]:
exposures_epl.new_net_exposure_gbp.sum()

-2206663.4251989024

### Step 7.1) Add the entity_ccy amount of the back-to-back from the internal and also the ones from the external

In [40]:
# Above
exposures_epl.loc[
    exposures_epl.index.get_level_values(0)=="GBP",
    ["new_above_exposure_gbp"]
] += (np.where(
    exposures_epl[["above_lhs_rhs"]]=="LHS",
    exposures_epl[["internal_change_gbp"]],
    -1 * exposures_epl[["internal_change_gbp"]]
).sum(axis=0) + np.where(
    exposures_epl[["above_to_epuk_lhs_rhs"]]=="LHS",
    exposures_epl[["above_to_epuk_gbp"]],
    -1 * exposures_epl[["above_to_epuk_gbp"]]
).sum(axis=0))

# Below
exposures_epl.loc[
    exposures_epl.index.get_level_values(0)=="GBP",
    ["new_below_exposure_gbp"]
] += (np.where(
    exposures_epl[["below_lhs_rhs"]]=="LHS",
    exposures_epl[["internal_change_gbp"]],
    -1 * exposures_epl[["internal_change_gbp"]]
).sum(axis=0))

# Net (only changes due to the external movement of exposure)
exposures_epl.loc[
    exposures_epl.index.get_level_values(0)=="GBP",
    ["new_net_exposure_gbp"]
] += np.where(
    exposures_epl[["above_to_epuk_lhs_rhs"]]=="LHS",
    exposures_epl[["above_to_epuk_gbp"]],
    -1 * exposures_epl[["above_to_epuk_gbp"]]
).sum(axis=0)


# Reshape it to the standard shape
exposures_epl = exposures_epl.reset_index()
exposures_epl["entity"] = "EPL"

exposures_epl = exposures_epl.set_index(["currency","entity","entity_ccy"]) 

In [41]:
exposures_epl.new_net_exposure_gbp.sum()

-90783264.64572896

# AHORA SOLO FALTA AÑADIR LAS EXPOSICIONES AL TOTAL, COMPROBAR QUE TODO ME QUEDE NETEADO, Y MOVER LOS HEDGES EXTERNOS DE EPL A EPUK

## Step 9) List all the hedges that take place:

In [42]:
hedges = pd.DataFrame()

The internal hedges for each entity (except EPUK and EPL)

In [43]:
print("Internal hedges: \n")
count = 0
for row in exposures_entities_net_6.iterrows():
    
    if row[1]["internal_change_gbp"] != 0 and row[0][1] not in ("EPUK","EPL"):
        count += 1
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_lhs_rhs'],
                "amount_gbp": row[1]['internal_change_gbp'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "entities_above_below"
            },
            orient='index').T
        ])
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['below_lhs_rhs'],
                "amount_gbp": row[1]['internal_change_gbp'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "BS FX Risk hedge",
                "type": "entities_above_below"
            },
            orient='index').T
        ])
        # print(f"{count}.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['above_lhs_rhs']}. Amount: {round(row[1]['internal_change_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: Revenue FX Risk hedge.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['below_lhs_rhs']}. Amount: {round(row[1]['internal_change_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: BS FX Risk hedge.")
        # print()
        

Internal hedges: 



In [44]:
hedges.shape

(52, 8)

The hedges to EPUK for each entity (except EPUK and EPL)

In [45]:
print("Internal hedges: \n")
count = 0
for row in exposures_entities_net_6.iterrows():
    
    if row[1]["above_to_epuk_gbp"] != 0 and row[0][1] not in ("EPUK","EPL"):
        count += 1
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_to_epuk_lhs_rhs'],
                "amount_gbp": row[1]['above_to_epuk_gbp'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "above_to_epuk"
            },
            orient='index').T
        ])
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": np.where(row[1]['above_to_epuk_lhs_rhs']=='RHS','LHS','RHS'),
                "amount_gbp": row[1]['above_to_epuk_gbp'],
                "entity": "EPUK",
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "above_to_epuk"
            },
            orient='index').T
        ])
        # print(f"{count}.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['above_to_epuk_lhs_rhs']}. Amount: {round(row[1]['above_to_epuk_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: Revenue FX Risk hedge.")
        # print(f"{row[0][0]}{row[0][2]} {np.where(row[1]['above_to_epuk_lhs_rhs']=='RHS','LHS','RHS')}. Amount: {round(row[1]['above_to_epuk_gbp']):.3e} GBP. Entity: EPUK. Dealset: Revenue FX Risk hedge.")
        # print()
        

Internal hedges: 



In [46]:
hedges.shape

(100, 8)

The internal hedges in EPUK

In [47]:
print("Internal hedges: \n")
count = 0
for row in exposures_entities_net_6.iterrows():
    
    if row[1]["internal_change_gbp"] != 0 and row[0][1] == "EPUK":
        count += 1
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_lhs_rhs'],
                "amount_gbp": row[1]['internal_change_gbp'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "epuK_above_below"
            },
            orient='index').T
        ])
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['below_lhs_rhs'],
                "amount_gbp": row[1]['internal_change_gbp'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "BS FX Risk hedge",
                "type": "epuK_above_below"
            },
            orient='index').T
        ])
        # print(f"{count}.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['above_lhs_rhs']}. Amount: {round(row[1]['internal_change_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: Revenue FX Risk hedge.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['below_lhs_rhs']}. Amount: {round(row[1]['internal_change_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: BS FX Risk hedge.")
        # print()
        

Internal hedges: 



In [48]:
hedges.shape

(116, 8)

The external hedges in EPUK for covering EPUK (except the entity ccys)

In [49]:
print("External hedges: \n")
count = 0
for row in exposures_entities_net_6.iterrows():
    
    if row[1]["above_to_epuk_gbp"] != 0 and row[0][1] == "EPUK":
        count += 1
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "external",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_to_epuk_lhs_rhs'],
                "amount_gbp": row[1]['above_to_epuk_gbp'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "external_epuk"
            },
            orient='index').T
        ])

        # print(f"{count}.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['above_to_epuk_lhs_rhs']}. Amount: {round(row[1]['above_to_epuk_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: Revenue FX Risk hedge.")
        # print()
        

External hedges: 



In [50]:
hedges.shape

(125, 8)

The internal hedges in EPL to cover the entity curencies

In [51]:
print("Internal hedges: \n")
count = 0
for row in exposures_epl.iterrows():
    
    if row[1]["internal_change_gbp"] != 0:
        count += 1
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_lhs_rhs'],
                "amount_gbp": row[1]['internal_change_gbp'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "epl_above_below_entity_currencies"
            },
            orient='index').T
        ])
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['below_lhs_rhs'],
                "amount_gbp": row[1]['internal_change_gbp'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "BS FX Risk hedge",
                "type": "epl_above_below_entity_currencies"
            },
            orient='index').T
        ])
        # print(f"{count}.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['above_lhs_rhs']}. Amount: {round(row[1]['internal_change_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: Revenue FX Risk hedge.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['below_lhs_rhs']}. Amount: {round(row[1]['internal_change_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: BS FX Risk hedge.")
        # print()
        

Internal hedges: 



The external hedges in EPL (booked by EPUK) and the corresponding movement from EPUK to EPL

In [52]:
print("External & internal hedges: \n")
count = 0
for row in exposures_epl.iterrows():
    
    if row[1]["above_to_epuk_gbp"] != 0:
        count += 1
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "external",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_to_epuk_lhs_rhs'],
                "amount_gbp": row[1]['above_to_epuk_gbp'],
                "entity": "EPUK",
                "entity_ccy": "GBP",
                "dealset": "Revenue FX Risk hedge",
                "type": "external_epl_in_epuk"
            },
            orient='index').T
        ])
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": np.where(row[1]['above_to_epuk_lhs_rhs']=='RHS','LHS','RHS'),
                "amount_gbp": row[1]['above_to_epuk_gbp'],
                "entity": "EPUK",
                "entity_ccy": "GBP",
                "dealset": "Revenue FX Risk hedge",
                "type": "internal_epuk_epl"
            },
            orient='index').T
        ])
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_to_epuk_lhs_rhs'],
                "amount_gbp": row[1]['above_to_epuk_gbp'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "internal_epuk_epl"
            },
            orient='index').T
        ])
        # print(f"{count}.")
        # print(f"(External) {row[0][0]}{row[0][2]} {row[1]['above_to_epuk_lhs_rhs']}. Amount: {round(row[1]['above_to_epuk_gbp']):.3e} GBP. Entity: EPUK. Dealset: Revenue FX Risk hedge.")
        # print(f"(Internal) {row[0][0]}{row[0][2]} {np.where(row[1]['above_to_epuk_lhs_rhs']=='RHS','LHS','RHS')}. Amount: {round(row[1]['above_to_epuk_gbp']):.3e} GBP. Entity: EPUK. Dealset: Revenue FX Risk hedge.")
        # print(f"(Internal) {row[0][0]}{row[0][2]} {row[1]['above_to_epuk_lhs_rhs']}. Amount: {round(row[1]['above_to_epuk_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: Revenue FX Risk hedge.")
        # print()
        

External & internal hedges: 



In [53]:
hedges = hedges.reset_index(drop=True)

In [54]:
hedges

,internal_external,pair,direction,amount_gbp,entity,entity_ccy,dealset,type
0,internal,EURHKD,LHS,629381,EPHK,HKD,Revenue FX Risk hedge,entities_above_below
1,internal,EURHKD,RHS,629381,EPHK,HKD,BS FX Risk hedge,entities_above_below
2,internal,USDHKD,LHS,3.4313e+06,EPHK,HKD,Revenue FX Risk hedge,entities_above_below
3,internal,USDHKD,RHS,3.4313e+06,EPHK,HKD,BS FX Risk hedge,entities_above_below
4,internal,GBPHKD,RHS,7.14002e+06,EPHK,HKD,Revenue FX Risk hedge,entities_above_below
...,...,...,...,...,...,...,...,...
155,internal,CHFGBP,LHS,3.99946e+06,EPUK,GBP,Revenue FX Risk hedge,internal_epuk_epl
156,internal,CHFGBP,RHS,3.99946e+06,EPL,GBP,Revenue FX Risk hedge,internal_epuk_epl
157,external,AUDGBP,RHS,1.79101e+06,EPUK,GBP,Revenue FX Risk hedge,external_epl_in_epuk
158,internal,AUDGBP,LHS,1.79101e+06,EPUK,GBP,Revenue FX Risk hedge,internal_epuk_epl


In [55]:
exposures_entities_net_6.loc[
    (np.abs(exposures_entities_net_6.new_above_exposure_gbp)>2*entity_threshold+1)|
    (np.abs(exposures_entities_net_6.new_below_exposure_gbp)>2*entity_threshold+1)|
    (np.abs(exposures_entities_net_6.new_net_exposure_gbp)>2*entity_threshold+1)
][[c for c in exposures_entities_net_6.columns if "ccy" not in c]]

,,,above_exposure_gbp,below_exposure_gbp,net_exposure_gbp,new_above_exposure_gbp,new_below_exposure_gbp,new_net_exposure_gbp,internal_change_gbp,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_lhs_rhs
currency,entity,entity_ccy,,,,,,,,,,,
USD,EPHK,HKD,-2.187770e+04,-3.578362e+06,-3.600239e+06,-2.941176e+05,-1.470588e+05,-4.411765e+05,3.431303e+06,LHS,RHS,3.159063e+06,RHS
GBP,EPHK,HKD,1.113629e+04,7.287082e+06,7.298219e+06,2.941176e+05,1.470588e+05,4.411765e+05,7.140024e+06,RHS,LHS,6.857042e+06,LHS
HKD,EPHK,HKD,1.847290e+04,-5.329014e+05,-5.144285e+05,6.371129e+05,2.546438e+06,3.183551e+06,0.000000e+00,NaN,NaN,0.000000e+00,NaN
USD,EPCA,CAD,1.995084e+06,4.386915e+06,6.381999e+06,2.941176e+05,1.470588e+05,4.411765e+05,4.239857e+06,RHS,LHS,5.940823e+06,LHS
EUR,EPCA,CAD,9.321314e+04,-8.011664e+05,-7.079532e+05,-2.941176e+05,-1.470588e+05,-4.411765e+05,6.541075e+05,LHS,RHS,2.667767e+05,RHS
GBP,EPCA,CAD,2.928840e+03,-3.036626e+06,-3.033698e+06,-2.941176e+05,-1.470588e+05,-4.411765e+05,2.889568e+06,LHS,RHS,2.592521e+06,RHS
CAD,EPCA,CAD,1.463505e+06,7.134691e+05,2.176974e+06,3.848849e+06,1.409651e+06,5.258499e+06,0.000000e+00,NaN,NaN,0.000000e+00,NaN
GBP,EPL,GBP,0.000000e+00,6.285352e+07,6.285352e+07,0.000000e+00,6.285352e+07,6.285352e+07,0.000000e+00,NaN,NaN,0.000000e+00,NaN
HKD,EPL,GBP,0.000000e+00,-3.064122e+06,-3.064122e+06,0.000000e+00,-3.064122e+06,-3.064122e+06,0.000000e+00,NaN,NaN,0.000000e+00,NaN


In [56]:
def get_config():
    config = ConfigParser()
    config.read(os.path.join("C:/Users/andres.mireles_ebury/Desktop/Projects/FX Exposure/fx_exposure/config.ini"))

    return config


In [57]:
config = get_config()
bq_client, _ = get_bq_link(config)

In [58]:
hedges["direction"] = hedges["direction"].astype(str)
hedges["amount_gbp"] = hedges["amount_gbp"].astype(float)

In [59]:
hedges.dtypes

internal_external     object
pair                  object
direction             object
amount_gbp           float64
entity                object
entity_ccy            object
dealset               object
type                  object
dtype: object

In [60]:
import upload

upload.upload_table(
    hedges,
    "hedges_entities",
    bq_client,
    config,
    balance_date
)

Uploading to `root-rarity-166622.portfolio_metrics.hedges_entities`...
Upload successful


In [61]:
# import matplotlib.pyplot as plt

# fig, axs = plt.subplots(1,2,figsize=(15,5))

# axs[0].bar(x=exposures_entities["above_exposure_gbp"].index.get_level_values(0),height=exposures_entities["above_exposure_gbp"])
# axs[0].bar(x=exposures_entities["below_exposure_gbp"].index.get_level_values(0),height=exposures_entities["below_exposure_gbp"])
# axs[0].legend(["Above Exposure (GBP)","Below Exposure (GBP)"])
# axs[0].set_title("Pre Exposure")
# axs[0].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)

# axs[1].bar(x=exposures_entities["new_above_gbp"].index.get_level_values(0),height=exposures_entities["new_above_gbp"])
# axs[1].bar(x=exposures_entities["new_below_gbp"].index.get_level_values(0),height=exposures_entities["new_below_gbp"])
# axs[1].legend(["New Above Exposure (GBP)","New Below Exposure (GBP)"])
# axs[1].set_title("New Exposure")
# axs[1].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)
# plt.tight_layout()

In [62]:
# import matplotlib.pyplot as plt

# fig, axs = plt.subplots(1,2,figsize=(15,5))

# axs[0].bar(x=exposures_entities["net_exposure_gbp"].index.get_level_values(0),height=exposures_entities["net_exposure_gbp"],color="green")
# axs[0].legend(["Net Exposure (GBP)"])
# axs[0].set_title("Pre Exposure")
# axs[0].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)

# axs[1].bar(x=exposures_entities["new_net_gbp"].index.get_level_values(0),height=exposures_entities["new_net_gbp"],color="green")
# axs[1].legend(["New Net Exposure (GBP)"])
# axs[1].set_title("New Exposure")
# axs[1].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)
# plt.show()